# LSTMを用いてUSDJPYの予測を行う。
ディープラーニングを用いてUSDJPYの値動きを予測したい。
時系列データを扱うRNNの一種であるLong Shot Term Memoryを使用する。

LSTM理論
https://qiita.com/KojiOhki/items/89cd7b69a8a6239d67ca

LSTM実装参考
https://qiita.com/sasayabaku/items/b7872a3b8acc7d6261bf

USDJPYヒストリカルデータ
https://www.m2j.co.jp/market/historical.php

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [3]:
df = pd.read_csv('USDJPY.csv')
df.head()

,日付,始値,高値,安値,終値
0,2007/04/02,117.84,118.08,117.46,117.84
1,2007/04/03,117.84,118.98,117.72,118.96
2,2007/04/04,118.92,119.08,118.56,118.72
3,2007/04/05,118.72,118.99,118.44,118.72
4,2007/04/06,118.72,119.39,118.67,119.27


In [4]:
df.columns = ['date','open', 'high', 'low', 'close']
price_cols = ['open', 'high', 'low', 'close']

In [5]:
df_mod  = df.copy()
df_mod[price_cols] = df_mod[price_cols] / df['open'][0]

ヒストリカルデータを全て学習に使ってしまうと、後で予測の検証が出来ないので適当な時点で分割しておく。

データの持ち方としては冗長になるが、ある時点でのTrainデータはあらかじめ決めておいたDurationの分の過去のデータを含むようにしておくと実装の見通しが良い。

In [6]:
def make_dataset(raw_data, duration):
    len_data = raw_data.shape[0]
    dim_data = raw_data.shape[1]
    data_list = []
    target_list =  []
    for i in range(len_data - duration):
        data_list.append(raw_data[i:i+duration])
        target_list.append(raw_data[i+duration])
        
    data = np.array(data_list).reshape([-1,  duration, dim_data])
    target = np.array(target_list).reshape(-1, dim_data)
    
    return data, target

In [7]:
def make_model(X, n_hidden=300):
    len_seq = X.shape[1]
    len_out = X.shape[2]

    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None, len_seq, len_out), return_sequences=False))
    model.add(Dense(len_out))
    model.add(Activation("linear"))
    optimizer = Adam(lr=0.001)
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    
    return model

def fit_model(model, X, y):
    early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=5)
    model.fit(X, y, batch_size=30, epochs=30, validation_split=0.1, callbacks=[early_stopping], verbose=0)
    return model

In [8]:
def predict_future(X_train, model, len_future):
    x = X_train[-1, :, :]
    duration = X_train.shape[1]
    dim = X_train.shape[2]
    pred_list = []
    for _ in range(len_future):
        pred_ = model.predict(x.reshape([1, duration, dim]))
        pred_list.append(pred_)
        x = np.vstack([x[1:], pred_])
        pred_feature = np.array(pred_list).reshape([-1, dim])
    return pred_feature

In [9]:
num_train = 1000
split_pos = 50
offset = 200
len_future = 3

vote_list = []
out_list = []

for i in range(100):
    offset = 200 - i
    prices = df_mod[price_cols].values[-(num_train + split_pos + offset):]
    train = prices[:num_train]
    test = prices[num_train:]

    model_list = []
    duration_list = [20, 20, 20, 20, 20]
    pred_list = []
    future_list = []

    vote = 0

    for duration in duration_list:
        X_train, y_train = make_dataset(train, duration)
        model = make_model(X_train, n_hidden=300)
#         model.summary()
        model = fit_model(model, X_train, y_train)
        future = predict_future(X_train, model, len_future=len_future)

        vote += (future[-1,0] - future[0,0] > 0) * 2 -1

        model_list.append(model)
        pred_list.append(model.predict(X_train))
        future_list.append(future)

    out = ((test[len_future] - train[-1])[0] > 0) * 2 -1
    if vote > 1:
        vote = 1
    elif vote < -1:
        vote = -1
    else:
        vote = 0
    vote_list.append(vote)
    out_list.append(out)

ResourceExhaustedError: OOM when allocating tensor with shape[300,1200] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_209/Adam/gradients/lstm_210/strided_slice_6_grad/StridedSliceGrad = StridedSliceGrad[Index=DT_INT32, T=DT_FLOAT, begin_mask=1, ellipsis_mask=0, end_mask=1, new_axis_mask=0, shrink_axis_mask=0, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_209/Adam/gradients/lstm_210/strided_slice_6_grad/Shape, lstm_210/strided_slice_1/stack_1, lstm_210/strided_slice_3/stack, lstm_210/strided_slice_3/stack_2, training_209/Adam/gradients/lstm_210/while/MatMul_6/Enter_grad/b_acc_3)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [10]:
vote_arr = np.array(vote_list)
out_arr = np.array(out_list)

sum((vote_arr * out_arr) > 0) / len(vote_arr)

0.4634146341463415

In [11]:
vote_arr

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  0,  1,  0,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1])

In [12]:
out_arr

array([ 1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,
       -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
        1, -1, -1, -1, -1, -1, -1])

In [ ]:
# plt.figure()
# plt.plot(range(0, len(prices)), prices[:,3], color='b', label='raw_data', marker='.')

# cmap = plt.get_cmap("tab10")
# for i, pred, duration, model, future  in zip(range(len(pred_list)), pred_list, duration_list, model_list, future_list):
#     plt.plot(range(duration, len(pred)+duration), pred[:,3], color=cmap(i), label='predicted_{}'.format(duration))
#     plt.plot(range(len(pred)+duration, len(pred)+duration+len(future)), future[:,3], color=cmap(i+6), label='predicted_future_{}'.format(duration))

# # plt.ylim(0.90, 0.96)
# plt.xlim(len(pred)+duration-50, len(pred)+duration+50)
# #plt.legend()    
# plt.show()